# Pre-processing

In [ ]:
import ibis
from ibis import _

conn = ibis.duckdb.connect("tmp", extensions=["spatial"])
ca_parquet = "https://data.source.coop/cboettig/ca30x30/ca_areas.parquet"
# or use local copy:
ca_parquet = "/home/rstudio/source.coop/cboettig/ca30x30/ca_areas.parquet"


In [ ]:
buffer = -2

tbl = (
    conn.read_parquet(ca_parquet)
    .cast({"SHAPE": "geometry"})
    .rename(geom = "SHAPE")
#    .filter(_.UNIT_NAME == "Angeles National Forest")
    .filter(_.reGAP < 3) 
)
tbl_2023 = tbl.filter(_.Release_Year == 2023).mutate(geom=_.geom.buffer(buffer))
tbl_2024 = tbl.filter(_.Release_Year == 2024)
intersects = tbl_2024.anti_join(tbl_2023, _.geom.intersects(tbl_2023.geom))



In [ ]:
## Testing, run only on subset data
if False:
    gdf = intersects.mutate(geom = _.geom.convert("epsg:3310","epsg:4326")).execute()
    gdf_2023 = tbl_2023.mutate(geom = _.geom.convert("epsg:3310","epsg:4326")).execute()
    gdf_2024 = tbl_2024.mutate(geom = _.geom.convert("epsg:3310","epsg:4326")).execute()
    # gdf = ca2024
    established = {'property': 'established',
                   'type': 'categorical',
                   'stops': [
                       [2023, "#26542C80"], 
                       [2024, "#F3AB3D80"]]
                  }
    inter = {"fill-color": "#F3AB3D"}
    p2024 = {"fill-color": "#26542C"}
    p2023 = {"fill-color": "#8B0A1A"}
    
    m = leafmap.Map(style="positron")
    m.add_gdf(gdf_2024,layer_type="fill", name = "2024", paint = p2024)
    m.add_gdf(gdf_2023,layer_type="fill", name = "2023", paint = p2023)
    m.add_gdf(gdf,layer_type="fill", name = "intersects", paint = inter)
    
    m.add_layer_control()
    m

In [ ]:
%%time

new2024 = intersects.select("OBJECTID").mutate(established = 2024)

ca = (conn
      .read_parquet(ca_parquet)
      .cast({"SHAPE": "geometry"})
      .mutate(area = _.SHAPE.area())
      .filter(_.Release_Year == 2024)
      .filter(_.reGAP < 3)
      .left_join(new2024, "OBJECTID")
      .mutate(established=_.established.fill_null(2023))
      .mutate(geom = _.SHAPE.convert("epsg:3310","epsg:4326"))
      .rename(name = "cpad_PARK_NAME", access_type = "cpad_ACCESS_TYP", manager = "cpad_MNG_AGENCY",
              manager_type = "cpad_MNG_AG_LEV", id = "OBJECTID", type = "TYPE")
      .select(_.established, _.reGAP, _.name, _.access_type, _.manager, _.manager_type,
              _.Easement, _.Acres, _.id, _.type, _.geom)
     )
ca2024 = ca.execute()



ca2024.to_parquet("ca2024.parquet")



In [2]:
ca2024 = conn.read_parquet("ca2024.parquet")
ca2024.execute().to_file("ca2024.geojson") # tippecanoe can't parse geoparquet :-(

In [ ]:
## Upload to Huggingface
# https://huggingface.co/datasets/boettiger-lab/ca-30x30/

from huggingface_hub import HfApi, login
import streamlit as st
login(st.secrets["HF_TOKEN"])
api = HfApi()

def hf_upload(file):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id="boettiger-lab/ca-30x30",
            repo_type="dataset",
        )
hf_upload("ca2024.parquet")

# Testing & visualization

`ca2024.parquet()` now contains all we need.  The code below illustrates some quick examples of the kinds of visualizations and summaries we might want to compute with this data.  


In [1]:
import leafmap.maplibregl as leafmap
import ibis
from ibis import _
conn = ibis.duckdb.connect(extensions=["spatial"])

ca2024 = conn.read_parquet("ca2024.parquet")

In [ ]:
# compute some summary tables:

(ca2024
 .filter(_.established == 2024)
 .filter(_.manager_type == "State")
 .group_by(_.manager, _.manager_type)
 .agg(area = _.Acres.sum())
 .order_by(_.area.desc())
 .execute()
)

In [ ]:

gdf = ca2024.execute()
established = {'property': 'established',
               'type': 'categorical',
               'stops': [
                   [2023, "#26542C80"], 
                   [2024, "#F3AB3D80"]]}
paint = {"fill-color": established}


m = leafmap.Map(style="positron")
m.add_gdf(gdf,layer_type="fill", name = "intersects", paint = paint)

m.add_layer_control()
m.to_html("ca2024.html")
m

In [ ]:
import subprocess
import os

def generate_pmtiles(input_file, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-z", str(max_zoom),
        "--drop-densest-as-needed",
        "--extend-zooms-if-still-dropping",
        "--force",
        input_file
    ]

    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")

generate_pmtiles("ca2024.geojson", "ca2024-tippe.pmtiles")
hf_upload("ca2024-tippe.pmtiles")